<a href="https://colab.research.google.com/github/SaketMunda/human-classifier-unsplash-dataset/blob/master/human_classifier_transfer_learning_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Human Classifier using Transfer Learning Feature extraction

This notebook is responsible for building our own transfer learning feature extraction model for human classification problem.

For transfer learning, we'll take two models from [`tensorflow_hub`](https://www.tensorflow.org/hub) and visualize the outputs of both the experiments using [`Tensorboard Playground`](https://www.tensorflow.org/tensorboard).  